In [27]:
import os
from softtek_llm.chatbot import Chatbot
from softtek_llm.models import OpenAI
from softtek_llm.cache import Cache
from softtek_llm.vectorStores import PineconeVectorStore
from softtek_llm.embeddings import OpenAIEmbeddings
from softtek_llm.schemas import Filter
from dotenv import load_dotenv

In [28]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY not found in .env file")

OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
if OPENAI_API_BASE is None:
    raise ValueError("OPENAI_API_BASE not found in .env file")

OPENAI_EMBEDDINGS_MODEL_NAME = os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME")
if OPENAI_EMBEDDINGS_MODEL_NAME is None:
    raise ValueError("OPENAI_EMBEDDINGS_MODEL_NAME not found in .env file")

OPENAI_CHAT_MODEL_NAME = os.getenv("OPENAI_CHAT_MODEL_NAME")
if OPENAI_CHAT_MODEL_NAME is None:
    raise ValueError("OPENAI_CHAT_MODEL_NAME not found in .env file")
    #Se configura nombre de variable apy_key
PINECONE_API_KEY = os.getenv("api_key")
if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY not found in .env file")
#Se configura nombre de variable environment
PINECONE_ENVIRONMENT = os.getenv("environment")
if PINECONE_ENVIRONMENT is None:
    raise ValueError("PINECONE_ENVIRONMENT not found in .env file")
#Se configura nombre de variable index
PINECONE_INDEX_NAME = os.getenv("index")
if PINECONE_INDEX_NAME is None:
    raise ValueError("PINECONE_INDEX_NAME not found in .env file")

In [29]:
vector_store = PineconeVectorStore(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
    index_name=PINECONE_INDEX_NAME,
)
embeddings_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_EMBEDDINGS_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
)
cache = Cache(
    vector_store=vector_store,
    embeddings_model=embeddings_model,
)
model = OpenAI(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_CHAT_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
    verbose=True,
)
filters = [
    Filter(
        type="DENY",
        case="ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.",
    )
]
chatbot = Chatbot(
    model=model,
    description="You are a very helpful and polite chatbot",
    filters=filters,
    cache=cache,
    verbose=True,
)

In [36]:
response = chatbot.chat(
    "Hello! My name is Jeff.",
    print_cache_score=True
    #se elimina namespace
)

Revision prompt: Considering this context:
	- user: When did the Titanic sink?
	- user: Hello! My name is Jeff.
	- assistant: Hello again, Jeff! How can I assist you today?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
Hello! My name is Jeff.
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: When did the Titanic sink?\n\t- user: Hello! My name is Jeff.\n\t- assistant: Hello again, Jeff! How can I assist you today?\n\nPlease review the prompt below

In [31]:
response

Response(message=Message(role='assistant', content='Hello Jeff! How can I assist you today?'), created=1695515859, latency=454, from_cache=False, model='gpt-35-turbo-16k', usage=Usage(prompt_tokens=80, completion_tokens=10, total_tokens=90), additional_kwargs={})

In [32]:
response = chatbot.chat(
    "What is my name?",
    print_cache_score=True
)

Revision prompt: Considering this context:
	- user: What is my name?
	- user: Hello! My name is Jeff.
	- assistant: Hello Jeff! How can I assist you today?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
What is my name?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: What is my name?\n\t- user: Hello! My name is Jeff.\n\t- assistant: Hello Jeff! How can I assist you today?\n\nPlease review the prompt below and answer with "yes" if it adheres to t

In [33]:
response

Response(message=Message(role='assistant', content='Your name is Jeff.'), created=1695515860, latency=319, from_cache=False, model='gpt-35-turbo-16k', usage=Usage(prompt_tokens=103, completion_tokens=5, total_tokens=108), additional_kwargs={})

In [37]:
response = chatbot.chat(
    "When did the Titanic sink?",
    print_cache_score=True
)

Revision prompt: Considering this context:
	- assistant: Hello again, Jeff! How can I assist you today?
	- user: Hello! My name is Jeff.
	- assistant: Hello, Jeff! How can I assist you today?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
When did the Titanic sink?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- assistant: Hello again, Jeff! How can I assist you today?\n\t- user: Hello! My name is Jeff.\n\t- assistant: Hello, Jeff! How can I assist you

In [ ]:
chatbot.cache.vector_store.delete(delete_all=True, namespace="chatbot-test")